In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
df_p = pd.read_csv('./Data/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv')

In [3]:
df_s = pd.read_csv('./Data/Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [4]:
df_s_location = df_s[['sensor_id', 'latitude', 'longitude']]

In [5]:
df_p_19 = df_p.loc[df_p['Year'] == 2019]
df_p_19_ja = df_p_19.loc[df_p_19['Month'] == 'January']
df_p_19_ju = df_p_19.loc[df_p_19['Month'] == 'July']

In [6]:
df_p_19_ja_g = df_p_19_ja.groupby(['Sensor_ID']).sum()
df_p_19_ja_g = pd.merge(df_p_19_ja_g, df_s_location, left_on="Sensor_ID", right_on="sensor_id").drop('sensor_id', axis=1)
HC_sum_ja = df_p_19_ja_g['Hourly_Counts'].sum()
df_p_19_ja_g.Hourly_Counts = (df_p_19_ja_g.Hourly_Counts / HC_sum_ja) * 10000

df_p_19_ju_g = df_p_19_ju.groupby(['Sensor_ID']).sum()
df_p_19_ju_g = pd.merge(df_p_19_ju_g, df_s_location, left_on="Sensor_ID", right_on="sensor_id").drop('sensor_id', axis=1)
HC_sum_ju = df_p_19_ju_g['Hourly_Counts'].sum()
df_p_19_ju_g.Hourly_Counts = (df_p_19_ju_g.Hourly_Counts / HC_sum_ju) * 10000

In [10]:
map_january = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 13) 

# Ensure you're handing it floats
df_p_19_ja_g['latitude'] = df_p_19_ja_g['latitude'].astype(float)
df_p_19_ja_g['longitude'] = df_p_19_ja_g['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df_ja = pd.DataFrame(columns=['latitude',
                    'longitude'])
for index, row in df_p_19_ja_g.iterrows():
    for index in range(0, int(row['Hourly_Counts'])):
        df_row = pd.DataFrame([[row['latitude'],row['longitude']]], columns=['latitude', 'longitude'])
        heat_df_ja = heat_df_ja.append(df_row, ignore_index = True)

# List comprehension to make out list of lists
heat_data_ja = [[row['latitude'],row['longitude']] for index, row in heat_df_ja.iterrows()]

# Plot it on the map
HeatMap(heat_data_ja, blur=10, radius=15).add_to(map_january)

# Display the map
map_january

In [11]:
map_july = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 13) 

# Ensure you're handing it floats
df_p_19_ju_g['latitude'] = df_p_19_ju_g['latitude'].astype(float)
df_p_19_ju_g['longitude'] = df_p_19_ju_g['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df_ju = pd.DataFrame(columns=['latitude',
                    'longitude'])
for index, row in df_p_19_ju_g.iterrows():
    for index in range(0, int(row['Hourly_Counts'])):
        df_row = pd.DataFrame([[row['latitude'],row['longitude']]], columns=['latitude', 'longitude'])
        heat_df_ju = heat_df_ju.append(df_row, ignore_index = True)

# List comprehension to make out list of lists
heat_data_ju = [[row['latitude'],row['longitude']] for index, row in heat_df_ju.iterrows()]

# Plot it on the map
HeatMap(heat_data_ju, blur=10, radius=15).add_to(map_july)

# Display the map
map_july